# Note:
## to visualize the plots download $housing.csv$ and this Jupyter Notebook

In [85]:
# libraries required for the project
import os
import tarfile
from six.moves import urllib
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np

%matplotlib inline 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


# 1. Frame the problem and look at the big picture
1. Define the objective in business terms.
2. How will your solution be used?
3. What are the current solutions/workarounds (if any)?
4. How should you frame this problem (supervised/unsupervised, online/offline,
etc.)?
5. How should performance be measured?
6. Is the performance measure aligned with the business objective?
7. What would be the minimum performance needed to reach the business objective?
8. What are comparable problems? Can you reuse experience or tools?
9. Is human expertise available?
10. How would you solve the problem manually?
11. List the assumptions you (or others) have made so far.
12. Verify assumptions if possible.


### In this example, let's pretend we are assigned the task of building a model of house prices, the "hello world!" of Machine Learning. For this project, we are assigned a dataset of housing prices in California.  





#### The first question we need to ask ourselves is: what is the purpose of building this model (1.1)? Usually the model itself is not the end goal, but it will be used for business purposes. Knowing this an advance will save us the embarassement (and company time) of building a model that does not provide the insights the team was expecting / needing (1.2). This information will help us decide on some specifics of the model / project: for instance, based on the company necessity, this could be an online model that continuosly recieves data an has to be monitored and re-trained on an hourly basis or an offline model, that will be monitored and trained again when the performance drops under a set threshold (1.4). In this case, since we are dealing with labeled data, we have a *supervised learning* task. It is a *regression* problem,since we are trying to make a prediction, and since we are dealing with multiple features, it is a *multivariate regression*. Now that we framed the problem we can choose a performance metric (1.5). Typical metrics for regression problems are RMSE (root mean squared error) and MAE (mean average error). RMSE indicates the standard deviation from the mean. If the data follows a normal distribution, an RMSE of 50k dollars means about 68% of the model predictions fall within 50k dollars of the actual value and 95% within $100k of the actual value. This metric is not robust against outliers: in the presence of many outliers it is preferrable to use MAE.

#### Sharing information with the team and consulting human experts (1.9,1.10) with precious knowledge on the field which could save us time when building our model (for instance establishing a column of our dataset does not provide useful information to our model).  

# 2. Get the Data
    
1. List the data you need and how much you need.
2. Find and document where you can get that data.
3. Check how much space it will take.
4. Check legal obligations, and get authorization if necessary.
5. Get access authorizations.
6. Create a workspace.
7. Get the data.
8. Convert the data to a format you can easily manipulate.
9. Ensure sensitive information is deleted or protected.
10. Sample a test set.

#### The next step is getting the data and preparing our workspace. Depennding on the nature of the data, we may need to make particular arrangements (2.3-5).

#### Before starting to manipulate the data, we need to make sure we have all the proper software installed. It is good practice to setup an environment if working on several models: this will allow on using different version of the same Python library on the same machine.

#### Finally, we can get the data (2.7). It is better to write custom functions and automate the process when possible (after the model is deployed, most of the time we need to periodically input more data to re train the model when its performance drops or to prevent it from dropping)

#### Once we have the data we need to transform it in a format we can easily manipulate(2.8), like a *Pandas* dataframe

In [86]:
hd = pd.read_csv('housing.csv')


#### The last step is to sample a test set and put it aside. We will pick some instances randomly, let's say 20% of the dataset. One way of doing so is by using a custom function:

In [87]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [88]:
train_set, test_set = split_train_test(hd, 0.2)


#### The shortcoming of such function is that if we use it again, it will generate a different set of random indices, hence two different datasets. We can overcome this problem using a sklearn fucntion called *train_test_split* and setting a *random_state*

In [89]:
ran_train_set, ran_test_set = train_test_split(hd, test_size=0.2, random_state=42)


#### In theory, a random split is enough to generate two different dataset. In practice, there is a big risk our sample is biased e.g. not all the categories are well represented. To solve this problem, we should have a look at the type of data we have in each column and their distributions. We can check the data types using either *.dtypes* or *.info*

In [90]:
# the info method will also show us null values
hd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


#### We have null values only in the $total-bedrooms$ features, we will deal with this later. All features are numerical, except *ocean_proximity*. We can explore the numerical data with *.describe* and the categorical with the *.value_counts* method. Also, by looking at the names of the columns, we now know our label column is $median-house-value$

In [91]:
hd['ocean_proximity'].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [92]:
hd.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


#### It seems like $median-income$ is not expressed in USD. Moreover, all the values are between 0.5 and 15. In this case we would contact the team that collect the data to understand why is this the case. For instance, in this dataset the $median-income$ was capped and then scaled. 
#### We can also plot a histogram of our numerical values:

In [ ]:

hd.hist(bins=50, figsize=(20,15))

#### From the histograms we learned that:
1. both $housing-median-age$ and $median-house-value$ where capped. This could be a problem, especially since the latter is our label: the model may learn prices never go above that limit. In a real working environment, we would have to check with the team that will use the results of our model if this is problem or not
2. the features are in very different scales, we will need to deal with this later 
3. many histograms are *heavy-tailed*. Some machine learning algorithms require we transform these features into a more bell-shaped distribution using a log-scale

#### Now that we have a better understanding of the data, we may decide to use a different technique for the train and test split, for instance we can do so that all the categorical variables in $ocean-proximity$ are proportioannly represented. 

#### Let's pretend we had the opportunity to talk to an expert and found out that the median income is an important feature to rpedict median house prices. We want to make sure all the various categories of incomes are well represented in the test set. We can do so by creating a categorical attribute for the income, for the sole purpose of doing a *stratified* split

In [93]:
hd["income_cat"] = np.ceil(hd["median_income"] / 1.5)
hd["income_cat"].where(hd["income_cat"] < 5, 5.0, inplace=True)

In [94]:
# we succesfully created a categorical variable with 5 categories
hd['income_cat'].value_counts()/len(hd)*100

3.0    35.058140
2.0    31.884690
4.0    17.630814
5.0    11.443798
1.0     3.982558
Name: income_cat, dtype: float64

In [95]:
#compare with random split
ran_train_set, ran_test_set = train_test_split(hd, test_size=0.2, random_state=15)
ran_test_set['income_cat'].value_counts()/len(ran_test_set)*100


3.0    35.368217
2.0    31.589147
4.0    17.732558
5.0    11.409884
1.0     3.900194
Name: income_cat, dtype: float64

In [96]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(hd, hd["income_cat"]):
    train_set = hd.loc[train_index]
    test_set = hd.loc[test_index]

In [97]:
metrics = pd.DataFrame()
metrics['Overall %'] = hd['income_cat'].value_counts()/len(hd)*100
metrics['Stratified %'] = test_set['income_cat'].value_counts()/len(test_set)*100
metrics['Random %'] = ran_test_set['income_cat'].value_counts()/len(ran_test_set)*100
metrics['Stratified % error'] = 100-(metrics['Stratified %']/metrics['Overall %']*100) 
metrics['Random % error'] = 100-(metrics['Random %']/metrics['Overall %']*100) 
metrics

,Overall %,Stratified %,Random %,Stratified % error,Random % error
3.0,35.058140,35.053295,35.368217,0.013820,-0.884467
2.0,31.884690,31.879845,31.589147,0.015195,0.926911
4.0,17.630814,17.635659,17.732558,-0.027480,-0.577082
5.0,11.443798,11.458333,11.409884,-0.127011,0.296359
1.0,3.982558,3.972868,3.900194,0.243309,2.068127


#### We can see from the above table that the stratified split gives us a better representation of all the categories, compared to the random split. Before moving foward we only need to remove the $income-cat$ column

In [98]:
for col in (train_set, test_set):
    col.drop(["income_cat"], axis=1, inplace=True)

# 3. Explore the data to gain insights

1. Create a copy of the data for exploration (sampling it down to a manageable size
if necessary).
2. Create a Jupyter notebook to keep a record of your data exploration.
3. Study each attribute and its characteristics:
• Name
• Type (categorical, int/float, bounded/unbounded, text, structured, etc.)• % of missing values
• Noisiness and type of noise (stochastic, outliers, rounding errors, etc.)
• Possibly useful for the task?
• Type of distribution (Gaussian, uniform, logarithmic, etc.)
4. For supervised learning tasks, identify the target attribute(s).
5. Visualize the data.
6. Study the correlations between attributes.
7. Study how you would solve the problem manually.
8. Identify the promising transformations you may want to apply.
9. Identify extra data that would be useful.
10. Document what you have learned.

#### When working on real projects, often the datasets are in the size of gigabytes. This can make the taks of exploring the data slow and tedious. An alternative is sampling the dataset (3.1). In this case we will just make a copy of the dataset to explore 

In [99]:
df = train_set
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16354 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   median_house_value  16512 non-null  float64
 9   ocean_proximity     16512 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.4+ MB


#### Since we have the cooridnates of the districts with their housing prices, we can visualize the data using a scatter plot:

In [ ]:
df.plot(kind='scatter', x='latitude', y='longitude', figsize=(12,12))

#### We can improve the quality of the plot using *transparency, size and color*. Having semi-transparent points will make the areas with higher density of data darker

In [ ]:
df.plot(kind='scatter', x='latitude', y='longitude', figsize=(12,12), alpha=0.1)

#### While the alpha value has to be a number, we can pass a column to both *size & colour*. In the next example, the size of the points will reflect the population, and the color the median house price:

In [ ]:
df.plot(kind='scatter', x='latitude', y='longitude', figsize=(12,12),alpha=0.4, s=df['population']/100,c='median_house_value',cmap=plt.get_cmap('jet'),colorbar=True)

#### Next, we study the correlations(3.6). The *corr()* function computes the standard correlation coefficient between all atttributes. This can help us find redundant features and with features has a linear correlation with the label. This is computationally expensive, so in the case of big datasets the matrix can be calculated only after sampling. The correlation coefficients are still relevant, provided that the sampled dataset was stratified correctly. We also need to be aware that a weak or absent linear correlation does not mean a non-linear correlation does not exist. 

In [100]:
c_matrix = df.corr()
c_matrix['median_house_value'].sort_values(ascending=False)

median_house_value    1.000000
median_income         0.687160
total_rooms           0.135097
housing_median_age    0.114110
households            0.064506
total_bedrooms        0.047689
population           -0.026920
longitude            -0.047432
latitude             -0.142724
Name: median_house_value, dtype: float64

#### We can visually spot correlations by simply plotting all the columns against each other. Since we have 9 columns we would end up with 81 plots, but in this case we will only focus on the 3 features most correlated with the label 

In [ ]:
corr_plot=['median_house_value','median_income','total_rooms','housing_median_age']
scatter_matrix(df[corr_plot],figsize=(12,12))

#### There is definitely a strong correlation between $median-income$ & $median-house-value$. We also notice the price cap we saw before, along with some other few lines:

In [ ]:
df.plot(kind='scatter',x='median_income',y='median_house_value', figsize=(12,12), alpha=0.1)

#### We may have to remove those lines (districts) to prevent the model from learning this weird patterns 

#### We can also apply some data transformations. If we look at the feature again, we have the total population and total number of bedrooms in a district:

In [ ]:
df.describe()

#### Since we have the number of households, we can engineer some more useful features, like population per household , total rooms per household and bedrooms/rooms ratio

In [101]:
df_ = df.copy()
df_['rooms'] = df_['total_rooms']/df_['households']
df_['people'] = df_['population']/df_['households']
df_['ratio'] = df_['total_bedrooms']/df_['total_rooms']
c_matrix = df_.corr()
c_matrix['median_house_value'].sort_values(ascending=False)

median_house_value    1.000000
median_income         0.687160
rooms                 0.146285
total_rooms           0.135097
housing_median_age    0.114110
households            0.064506
total_bedrooms        0.047689
people               -0.021985
population           -0.026920
longitude            -0.047432
latitude             -0.142724
ratio                -0.259984
Name: median_house_value, dtype: float64

#### The new feature $ratio$ has the second strongest correlation with our label. Wuth these information we can move with more confidence to the next step. Note that often times this is not the end of the data exploration regarding to the project. We may decide to come back to this step after we completed our model and analysed our output, in order to gain more insights.

# 4. Prepare the Data

1. Data cleaning: fix or remove outliers (optional), fill in missing values (e.g., with zero, mean, median…) or drop their rows (or columns).
2. Feature selection (optional): Drop the attributes that provide no useful information for the task.
3. Feature engineering, where appropriate: discretize continuous features, decompose features (e.g., categorical, date/time, etc.)
4. Feature scaling: standardize or normalize features.


#### Now it's time to prepare the data. We will write custom functions to apply the transformations. This will allow us:
1. automatically transform other dataset (e.g. the test data we set aside and new training data in the future)
2. re-use the functions in other projects
3. treat the transformations as hyperparameters

#### From what we noticed so far, we need to:

1. deal with the missing values (we had them in $total-bedrooms$)
1. deal with the categorical feature $ocean-proximity$
1. add custom features
1. deal with the features different scales
1. use the log of the features with heavy-tailed distribution
1. deal with the lines in the $median-income$/$median-house-value$



In [102]:
df = train_set.drop("median_house_value", axis=1)
df_labels = train_set["median_house_value"].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16354 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   ocean_proximity     16512 non-null  object 
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


#### We have 4 options for the missing values:
1. drop all the rows with missing values with *dropna*
2. drop all the columns with missing values with *drop*
3. set the missing values to some value (e.g. the median) using *Scikit-Learn Imputer* 
4. stratified the dataset and apply method 3 to each subset

#### We will proceed with method n.3

In [103]:
imputer = SimpleImputer(strategy='median')
df_num = df.drop('ocean_proximity',axis=1)
imputer.fit(df_num)
imputer.statistics_
X = imputer.transform(df_num)
df_tr = pd.DataFrame(X, columns=df_num.columns)


In [104]:
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
dtypes: float64(8)
memory usage: 1.0 MB


2. deal with the categorical feature $ocean-proximity$

#### One approcah is using a label encoder, which will convert every unique category into a number. In this case we cannot use it, as machine learning models will imply there is an order in the numbers, e.g. category 5 is closer to category 4 then it is to category 1, which is not the case. A better approch will be using one-hot encoding

In [105]:
oh = LabelBinarizer()
df_cat = df[['ocean_proximity']]
df_oh = oh.fit_transform(df_cat)
df_oh

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       ...,
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0]])

#### 3. add custom features
#### For this step we will need to create a class. We can use the creation of bedrooms / rooms ratio as an hyperparameter


In [106]:
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
class Adder(BaseEstimator, TransformerMixin):
    def __init__(self, ratio = True): 
        self.ratio = ratio
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X, y=None):
        rooms = X[:, rooms_ix] / X[:, household_ix]
        people = X[:, population_ix] / X[:, household_ix]
        if self.ratio:
            ratio = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms, people, ratio]
        else:
            return np.c_[X, rooms, people]


In [107]:
attr_adder = Adder(ratio=True)
tr_extra = attr_adder.transform(df_tr.values)
df_tr_extra =  pd.DataFrame(tr_extra)
df_tr_extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       16512 non-null  float64
 1   1       16512 non-null  float64
 2   2       16512 non-null  float64
 3   3       16512 non-null  float64
 4   4       16512 non-null  float64
 5   5       16512 non-null  float64
 6   6       16512 non-null  float64
 7   7       16512 non-null  float64
 8   8       16512 non-null  float64
 9   9       16512 non-null  float64
 10  10      16512 non-null  float64
dtypes: float64(11)
memory usage: 1.4 MB


#### The latest dataframe has no missing values, because before creating the new features we filled the missing values in the original features using *Imputer*. The next step is feature scaling. We decided to use *standardization*, which is more robust against the outliers. We will integrate it in a pipeline

In [108]:
num_preprocess = Pipeline([
('missing_values', SimpleImputer(strategy="median")),
('feature_eng', Adder(ratio=True)),
('standardization', StandardScaler()),
])


In [109]:
df_num_tr = num_preprocess.fit_transform(df_num)

#### We will ignore point 5 and 6 for now (use the log of the features with heavy-tailed distribution, deal with the lines in the $median-income$/$median-house-value$). We can return on these later if we notice our model is not performing well. The last step is merging the one-hot encoding process in the pipeline. We will use *FeatureUnion*. Since we are working with two parts of the dataframe separately, we need to define a method to select the part we want to pass to each pipeline.


In [110]:
num_ = list(df_num)
cat_ = ['ocean_proximity']

full_pipeline = ColumnTransformer([
        ("num", num_preprocess, num_),
        ("cat", OneHotEncoder(), cat_),
    ])



In [112]:
df_ready = full_pipeline.fit_transform(df)

# 5. Short-List Promising Models

1. Train many quick and dirty models from different categories (e.g., linear, naive Bayes, SVM, Random Forests, neural net, etc.) using standard parameters.
2. Measure and compare their performance (for each model, use N-fold cross-validation and compute the mean and standard deviation of the performance measure on the N folds)
3. Analyze the most significant variables for each algorithm.
4. Analyze the types of errors the models make (what data would a human have used to avoid these errors?)
5. Have a quick round of feature selection and engineering.
6. Have one or two more quick iterations of the five previous steps.
7. Short-list the top three to five most promising models, preferring models that make different types of errors.

#### Now that the training dataset is ready, it is time to train our first models and short-list the best ones. We will start with multivariate linear regression

In [113]:
rg = LinearRegression()
rg.fit(df_ready, df_labels)
rg_pred = rg.predict(df_ready) 
rg_mse = mean_squared_error(df_labels,rg_pred)
rg_rmse = np.sqrt(rg_mse)
rg_rmse

68628.19819848923

#### This means that the typical prediciton error is around 68k dollars, which is not great since most of the $median-house-values$ range between 120k and 256k. This means the model is underfitting the training data. We can try to:
1. Use a more powerful model
2. feed the model better features (e.g. we could try the log of the heavy-tailed features)
3. remove regularisation
#### Since we haven't regularised the model yet we have to rule out number 3. Before going back to feature engineering (option 2), we can try a different model, like decision tree, SVM and random forest.

In [114]:
#decision tree regressor
dec_tr = DecisionTreeRegressor()
dec_tr.fit(df_ready,df_labels)
tr_pred = dec_tr.predict(df_ready)
tr_mse = mean_squared_error(df_labels,tr_pred)
tr_rmse = np.sqrt(tr_mse)
tr_rmse


0.0

In [115]:
# support vector machine regressor
svr = SVR(kernel='rbf', C=100, gamma=1)
svr.fit(df_ready,df_labels)
svr_pred = svr.predict(df_ready)
svr_mse = mean_squared_error(df_labels,svr_pred)
svr_rmse = np.sqrt(svr_mse)
svr_rmse

114998.48948921208

# 6. Fine-tune your models and combine them into a great solution

1. Fine-tune the hyperparameters using cross-validation: treat your data transformation choices as hyperparameters, unless there are very few hyperparameter values to explore, prefer random search over grid search. 
2. Try Ensemble methods. 
3. Once you are confident about your final model, measure its performance on the test set to estimate the generalization error.


#### The decision tree has zero error, which means the model is so good that fits all the training data, but it's not very likely to perform well on unseen data (overfitting). Since we cannot validat the result on the test set (yet) we can use a part of the training set to perform cross validation. In this case we will perform *K-fold cross validation*


In [116]:
tr_scores = cross_val_score(dec_tr, df_ready, df_labels, scoring = 'neg_mean_squared_error', cv=10)
tr_rmse_score = np.sqrt(-tr_scores)

In [117]:
def display_scores(scores):
    print('scores: ', scores)
    print('scores average: ', scores.mean())
    print('scores std dev: ', scores.std())

In [118]:
display_scores(tr_rmse_score)

scores:  [69144.15440697 67254.9325584  69668.17230368 68876.35818139
 69322.86772155 74377.35882996 70117.72010672 71788.84524801
 75381.91425275 70185.38177443]
scores average:  70611.77053838593
scores std dev:  2402.7252609932557


#### We can apply cross validation to the linear regression too, in order to compare the results:

In [119]:
lr_scores = cross_val_score(rg, df_ready, df_labels, scoring = 'neg_mean_squared_error', cv=10)
lr_rmse_score = np.sqrt(-lr_scores)
display_scores(lr_rmse_score)

scores:  [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
scores average:  69052.46136345083
scores std dev:  2731.6740017983425


#### So the decision tree is actually not performing as well as linear regression. Let's move to the next model: SVM. SVM does not seeem to be performing well, but we can try using different combination of the hyperparameters through a grid search. In this case we are trying a very small number of hyperparameters, but when the hyperparameters space becomes very large it is best to use a random search.

In [120]:
svm_param = [
    {'kernel': ['rbf'], 'C': [30, 100, 300], 'gamma': [0.3,1,3,10]}
    ]
svm_grid = GridSearchCV(svr, svm_param, cv=5, scoring='neg_mean_squared_error')
svm_grid.fit(df_ready,df_labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=100, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma=1, kernel='rbf', max_iter=-1,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [30, 100, 300], 'gamma': [0.3, 1, 3, 10],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [121]:
svm_grid.best_estimator_

SVR(C=300, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.3,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [122]:
svm_res = svm_grid.cv_results_
for mean,score in zip(svm_res['mean_test_score'],svm_res['params']):
    print(np.sqrt(-mean),score)

114592.76534069762 {'C': 30, 'gamma': 0.3, 'kernel': 'rbf'}
117945.46989557479 {'C': 30, 'gamma': 1, 'kernel': 'rbf'}
118773.67820979048 {'C': 30, 'gamma': 3, 'kernel': 'rbf'}
118914.60851572135 {'C': 30, 'gamma': 10, 'kernel': 'rbf'}
106517.78628279532 {'C': 100, 'gamma': 0.3, 'kernel': 'rbf'}
115842.43140426329 {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
118405.21796830831 {'C': 100, 'gamma': 3, 'kernel': 'rbf'}
118888.28605400377 {'C': 100, 'gamma': 10, 'kernel': 'rbf'}
95252.79103138024 {'C': 300, 'gamma': 0.3, 'kernel': 'rbf'}
110716.22044353209 {'C': 300, 'gamma': 1, 'kernel': 'rbf'}
117424.91641792165 {'C': 300, 'gamma': 3, 'kernel': 'rbf'}
118788.61693816051 {'C': 300, 'gamma': 10, 'kernel': 'rbf'}


#### The grid search suggests the lower the *gamma* is, the better our error gets. We can try performing another grid search with lower values of *gamma*. We can also try using a different kernel, like *linear*

In [128]:
svm_param_2 = [
    {'kernel': ['rbf'], 'C': [300,1000], 'gamma': [0.003,0.01,0.03,0.1]},
    {'kernel': ['linear'], 'C': [30,100,300,1000]}
    ]
svm_grid_2 = GridSearchCV(svr, svm_param_2, cv=5, scoring='neg_mean_squared_error')
svm_grid_2.fit(df_ready,df_labels)

svm_res_2 = svm_grid_2.cv_results_

In [129]:
svm_grid_2.best_estimator_

SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1,
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [130]:
for mean,score in zip(svm_res_2['mean_test_score'],svm_res_2['params']):
    print(np.sqrt(-mean),score)

108767.83105615216 {'C': 300, 'gamma': 0.003, 'kernel': 'rbf'}
95347.86986187777 {'C': 300, 'gamma': 0.01, 'kernel': 'rbf'}
84641.68341091352 {'C': 300, 'gamma': 0.03, 'kernel': 'rbf'}
84514.70619517785 {'C': 300, 'gamma': 0.1, 'kernel': 'rbf'}
93145.98873720676 {'C': 1000, 'gamma': 0.003, 'kernel': 'rbf'}
78423.96918256629 {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
72115.92421364697 {'C': 1000, 'gamma': 0.03, 'kernel': 'rbf'}
71925.25004187765 {'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}
75448.84743960595 {'C': 30, 'kernel': 'linear'}
71603.12196479437 {'C': 100, 'kernel': 'linear'}
70703.95891598675 {'C': 300, 'kernel': 'linear'}
70445.41077712944 {'C': 1000, 'kernel': 'linear'}


#### With this grid search, it does not seem we do better than linear regression. Let's try using grid search on the random forest regressor

In [123]:
fr_param = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
fr = RandomForestRegressor()
fr_grid = GridSearchCV(fr, fr_param, cv=5,scoring='neg_mean_squared_error')
fr_grid.fit(df_ready, df_labels)

fr_res = fr_grid.cv_results_


In [124]:
for mean,score in zip(fr_res['mean_test_score'],fr_res['params']):
    print(np.sqrt(-mean),score)

63936.32731940089 {'max_features': 2, 'n_estimators': 3}
55457.80944226246 {'max_features': 2, 'n_estimators': 10}
52806.23379242126 {'max_features': 2, 'n_estimators': 30}
59961.1193768923 {'max_features': 4, 'n_estimators': 3}
52881.54320070661 {'max_features': 4, 'n_estimators': 10}
50798.6131072422 {'max_features': 4, 'n_estimators': 30}
58282.29413841001 {'max_features': 6, 'n_estimators': 3}
52624.200333226436 {'max_features': 6, 'n_estimators': 10}
50144.98860348818 {'max_features': 6, 'n_estimators': 30}
58426.608030497635 {'max_features': 8, 'n_estimators': 3}
51637.837535344275 {'max_features': 8, 'n_estimators': 10}
49997.87939056066 {'max_features': 8, 'n_estimators': 30}
63470.44804226408 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54157.720743395934 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59925.435380061885 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52346.44930721333 {'bootstrap': False, 'max_features': 3, 'n_estimators':

In [125]:
fr_grid.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=8, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=30,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

#### The last thing left to do before testing our model is learn which features are most relevant:

In [126]:
feat_imp_fr = fr_grid.best_estimator_.feature_importances_
extra_attr = ['rooms','people','ratio']
cat_one_hot = list(oh.classes_)
attributes = num_ + extra_attr + cat_one_hot
sorted(zip(feat_imp_fr,attributes),reverse=True)

[(0.3674729601098822, 'median_income'),
 (0.16364037016063535, 'INLAND'),
 (0.11514070853074168, 'people'),
 (0.06479410509927182, 'longitude'),
 (0.06313077461661126, 'latitude'),
 (0.055183637284620765, 'ratio'),
 (0.0545696011320957, 'rooms'),
 (0.04524291906650208, 'housing_median_age'),
 (0.016292189026623168, 'population'),
 (0.015087006082926797, 'total_bedrooms'),
 (0.014782782897478797, 'total_rooms'),
 (0.014261401945697501, 'households'),
 (0.004309158393240756, '<1H OCEAN'),
 (0.003629324517849334, 'NEAR OCEAN'),
 (0.002352734588069074, 'NEAR BAY'),
 (0.000110326547753713, 'ISLAND')]

#### In a real project we would have the option to go back and remove some of the less relevant features, then train the model again and see if its performance improved (for instance, of the $ocean_proximity$ feature, only the *inland* category seems relevant). For the sake of this exercise, we would just assume this our final model and move to the last step: evaluate it on the test set.

In [127]:
model = fr_grid.best_estimator_

X_test = test_set.drop('median_house_value', axis = 1)
y_test = test_set['median_house_value'].copy()

X_test_ready = full_pipeline.transform(X_test)

predictions = model.predict(X_test_ready)
mse = mean_squared_error(y_test,predictions)
rmse = np.sqrt(mse)
rmse

48412.374859911215

#### Usually the test error is worse than the one on the validation set. It did not happen in this specific project, but when it happens it is important to not use this metric to tweak the hyperparameters until the test error improves, as this will only overfit the model on the test set.

# 7. Present your solution

1. Document what you have done.
2. Create a nice presentation: make sure you highlight the big picture first.
3. Explain why your solution achieves the business objective.
4. Don’t forget to present interesting points you noticed along the way: describe what worked and what did not, list your assumptions and your system’s limitations.
5. Ensure your key findings are communicated through beautiful visualizations or easy-to-remember statements (e.g., “the median income is the number-one predictor of housing prices”).


# 8. Launch, monitor, and maintain your system

1. Get your solution ready for production 
2. Write monitoring code to check your system’s live performance at regular intervals and trigger alerts when it drops. Beware of slow degradation
3. Retrain your models on a regular basis on fresh data (automate as much as possible).